<a href="https://colab.research.google.com/github/alialaki/Applied-AI-ML-for-HR/blob/main/AI%26HR_Code_04_Recommend_Course.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#User item Recommendations


*   Collaborative filtering technique
*   Similar users based on items that are common


*   Recommend items that similar users liked
*   Deep learning with embeddings




One of the key areas of focus for HR is to identify the right training and development programs, for each of their employees. IN this colab, we discuss specific technique, to recommend training programm to empolyees.


*   Employees need to training based on skill gaps-both current work and next level
*   List item

Training generates self-confidence, employee satisfaction and reduces attrition

*   HR organization needs to identify gaps and provide training on a continuous basis

#Training needs identified based on


*   Employee input
*   Supervisor input

*   Job role requirements

Lastly, these require information collection and consolidation.

Another way to do this is to look at training programs that are well-liked by similar peers. AI can help analyze ratings provided by peers to different training programs and choose the programs that a specific employee may like









#Word Embedding


*   Language modeling with feature learning
*   Text converted to numbers

*   Relationships between words represented by numeric values
*   Relationship scores better if used in similar context


*   In this use case employees and courses become words








In [1]:
from google.colab import drive
drive.mount('/content/drive')
import sys
import numpy as np
import pandas as pd
import math

Mounted at /content/drive


In [2]:
ratings_data = pd.read_csv('/content/drive/MyDrive/Applied AI for Human Resources-LinkedinLearning/employee_course_ratings.csv')
ratings_data.head()

,EmployeeID,EmpName,CourseID,CourseName,Rating
0,1408,Ignace Ormonde,14,Video Production,3
1,1249,Gabriela Balcon,17,Translation,2
2,1158,Enrique Lewer,8,IT Architecture,3
3,1564,Wallie Byrd,18,Natural Language Processing,3
4,1334,Hannah Ganter,6,Java Programming,4


In [14]:
# employees_over_2000 = rating_data[rating_data['EmployeeID'] > 1500]

# if not employees_over_2000.empty:
#     print("Employee IDs greater than 2000:")
#     display(employees_over_2000)
# else:
#     print("There are no employee IDs greater than 2000.")

In [3]:
#Build list of unique Employees
emp_list = ratings_data.groupby(['EmployeeID', 'EmpName']).size().reset_index()
emp_list.head()
print(f"Total Employees: {len(emp_list)}")

#Build list of unique courses
course_list = ratings_data.groupby(['CourseID', 'CourseName']).size().reset_index()
course_list.head()
print(f"Total Courses: {len(course_list)}")

Total Employees: 638
Total Courses: 25


In [4]:
from tensorflow.keras.layers import Input, Embedding, Flatten, Dot, Dense, Concatenate
from tensorflow.keras.models import Model

#build employee embedding vector
#we are using ID as the direct index to embedding
#Since IDs are continuous, we dont need ID-name mapping.
#We can also build a vocabulary alternatively.
emp_input = Input(shape=[1], name='Emp-Input')
emp_emb = Embedding(2001, #max value of employee ID
                    5,
                    name='Emp-Embedding')(emp_input)
emb_vec = Flatten(name='Flatten-Employees')(emp_emb)

#build course embedding vector
course_input = Input(shape=[1], name='Course-Input')
course_emb= Embedding(len(course_list) + 1,
                      5,
                      name='Course-Embedding')(course_input)
course_vec = Flatten(name='Flatten-Courses')(course_emb)

merged_vec = Concatenate()([emb_vec, course_vec])

#**04.05. Building the Keras Rating Model**
**The recommendation works as follows**


1.   Build a model that can predict the rating, a given employee may give to a course he/she has not taken so far.
2.   Use the model to predict possible ratings for all courses, for this employee.


1.   Recommend the courses that havce the top predicted ratings.







In [28]:
#Given an Employee and a Course, this model predicts the
#rating the employee will give this course

from sklearn.model_selection import train_test_split

ratings_train, ratings_test = train_test_split(ratings_data, test_size=0.1)

#add fully connected layers
fc_layer1 = Dense(128, activation='relu')(merged_vec)
fc_layer2 = Dense(64, activation='relu')(fc_layer1)
model_output = Dense(1)(fc_layer2)

rating_model = Model([emp_input,course_input], model_output)

rating_model.compile(optimizer='adam',
                     loss="mse")

rating_model.summary()

print("Fitting the model:")
#Fit the model
model_fit = rating_model.fit(
    x=[ratings_train.EmployeeID, ratings_train.CourseID],
    y=ratings_train.Rating,
    epochs=15,
    verbose=1,
    validation_split=0.1
    )

#Evalute the model
print("Evaluating the model:")
rating_model.evaluate(
    x=[ratings_test.EmployeeID, ratings_test.CourseID],
    y=ratings_test.Rating
    )



Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ Emp-Input           │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Course-Input        │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Emp-Embedding       │ (None, 1, 5)      │     10,005 │ Emp-Input[0][0]   │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Course-Embedding    │ (None, 1, 5)      │        130 │ Course-Input[0][… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Flatten-Employees   │ (None, 5)         │          0 │ Emp-Embedding[0]… │
│ (Flatten)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Flatten-Courses     │ (None, 5)         │          0 │ Course-Embedding… │
│ (Flatten)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 10)        │          0 │ Flatten-Employee… │
│ (Concatenate)       │                   │            │ Flatten-Courses[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_15 (Dense)    │ (None, 128)       │      1,408 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_16 (Dense)    │ (None, 64)        │      8,256 │ dense_15[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_17 (Dense)    │ (None, 1)         │         65 │ dense_16[0][0]    │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 19,864 (77.59 KB)

 Trainable params: 19,864 (77.59 KB)

 Non-trainable params: 0 (0.00 B)

Fitting the model:
Epoch 1/15
26/26 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 8.2559 - val_loss: 5.2815
Epoch 2/15
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 3.2248 - val_loss: 1.0620
Epoch 3/15
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.9690 - val_loss: 0.8485
Epoch 4/15
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.5727 - val_loss: 0.7778
Epoch 5/15
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.4634 - val_loss: 0.7947
Epoch 6/15
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.4131 - val_loss: 0.7916
Epoch 7/15
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.4014 - val_loss: 0.8041
Epoch 8/15
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.3702 - val_loss: 0.7938
Epoch 9/15
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.3567 - val_loss: 0.7708
Epoch 10/15
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.2963 - val_loss: 0.7782
Epoch 11/15
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.3706 - val_loss: 0.7550
Epoch 12/15
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - l

0.7274309396743774

#**04.06. Recommending Courses with Keras**

In [29]:
#Predicting the Rating for a given employee and a course
#for employee 1029 and course 8

rating_model.predict([pd.Series([1029]), pd.Series([8])])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step


array([[3.4707854]], dtype=float32)

In [30]:
emp_to_predict = "Harriot Laflin"

#Get employee ID for the employee name
pred_emp_id = emp_list[emp_list.EmpName == emp_to_predict].EmployeeID.iloc[0]
#print(pred_emp_id)

#find Courses already taken by employee. We don't want to predict those
completed_courses = ratings_data[ratings_data.EmployeeID == pred_emp_id].CourseID.unique()
#print(completed_courses)

#Courses not taken by employee
new_courses = course_list.query("CourseID not in @completed_courses")["CourseID"]
#print(new_courses)

#Create a list with same employee ID repeated for the same number of times as the
#number of new courses. This provides the employee and course Series with same size
emp_dummy_list = pd.Series(np.array([pred_emp_id for i in range(len(new_courses))]))

#Predict ratings for the new courses for this employee
projected_ratings = rating_model.predict([emp_dummy_list, new_courses])
flat_ratings = np.array([x[0] for x in projected_ratings])
print(f"Course Ratings: {flat_ratings}")

#Recommend top 5 courses
print("\nRating CourseID CourseName\n--------------------------")
for idx in (-flat_ratings).argsort()[:5]:
  course_id=new_courses.iloc[idx]
  course_name=course_list.query("CourseID == @course_id")["CourseName"].iloc[0]
  print(" ", round(flat_ratings[idx],1), "   ", course_id, "   ", course_name)
  # print(f"{flat_ratings[idx]} {new_courses.iloc[idx]} {course_list[course_list.CourseID == new_courses.iloc[idx]].CourseName.iloc[0]}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 143ms/step
Course Ratings: [3.6721942 3.6653314 4.8104243 3.9657693 3.7011938 4.4442077 4.198623
 4.6214957 4.0479875 4.3220043 3.9074204 3.4926624 3.4835443 3.6643066
 4.304798  4.338974  3.4906907 3.859042  3.9616375 3.7670853 4.2073154
 4.301108  3.8511753]

Rating CourseID CourseName
--------------------------
  4.8     3     Data Management
  4.6     9     Blockchain
  4.4     7     Project Management
  4.3     17     Translation
  4.3     11     Analytical Reasoning
